In [1]:
import sys
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score, make_scorer
from imblearn.over_sampling import RandomOverSampler
from sklearn import model_selection
from sklearn.svm import SVC

#Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

sys.path.insert(0, '/Users/gracewang/Documents/GitHub/elecfinal')
sys.path.insert(0, 'D:\Fall23 Coursework\ELEC478\Competition\elecfinal')

In [2]:
from ml_pipeline import train_n_predict, validation, clean_split

In [3]:
from Data.data_cleaner import cleaner

In [4]:
## Clean data


train_path = "../Data/train_data.csv"
feature_path = "../Data/feature_weights.csv"
morph_path = "../Data/imputed_morph_embed_vector.csv"
X_train, X_val, X_query, y_train, y_val, y_query = clean_split(train_path, feature_path, morph_path)

In [25]:
from sklearn.manifold import TSNE

model = TSNE(n_components=2, perplexity = 50)
X_val2 = model.fit_transform(X_val.select_dtypes(include='number').drop("ID", axis = 1))


In [27]:
X_val2 = pd.DataFrame(X_val2)
X_val2.to_csv('tsne_X_val.csv')

In [28]:
valid_X = pd.read_csv('tsne_X_val.csv')
valid_y = y_val
valid_X=valid_X[["0", "1"]]
y_val

4418      False
4419      False
4420      False
4421      False
4422      False
          ...  
145434    False
145435    False
145436    False
145437    False
145438    False
Name: connected, Length: 37213, dtype: bool

Using Validation

In [29]:
"""
    Function that outputs a model with optimal hyperparameters
    based on a validation set using grid search

    Inputs:
    model: provided model
    param_grid: dictionary of parameters and values to validate on
    e.g. 
    {'C': [0.001,0.01,0.1,1,10], 
    'gamma':[0.1,1,10,100], 
    'kernel':('linear', 'rbf')}
    valid_X: validation X of data (pandas df)
    valid_y: validation y of data

    Outputs: 
    clf: provided model with optimum hyperparameters
    """
pre_valid_models = [RandomForestClassifier(), LinearDiscriminantAnalysis()]
param_grids = [
    {
    'max_features' : ['sqrt', 'log2'],
    'n_jobs' : [-1],
    'random_state': [1]
    },
    {
    'solver' : ['lsqr', 'eigen'],
    'shrinkage' : ['auto']
    }]
post_valid_models = []

for i in range(len(pre_valid_models)):
    best_clf = validation(model = pre_valid_models[i], 
                                        param_grid = param_grids[i], 
                                        valid_X = valid_X, 
                                        valid_y = valid_y)
    post_valid_models.append(best_clf)


USING ALL SVDs

In [30]:
pre_valid_models = [SVC()]
param_grids = [
    {
    'kernel':['rbf'],
    'C': [0.1, 1],
    }]
post_valid_models = []

for i in range(len(pre_valid_models)):
    best_clf = validation(model = pre_valid_models[i], 
                                        param_grid = param_grids[i], 
                                        valid_X = valid_X, 
                                        valid_y = valid_y)
    post_valid_models.append(best_clf)

In [31]:
post_valid_models[0]

SVC(C=0.1)

Training and Predicting

In [32]:
query_X = pd.read_csv('tsne_X_query.csv')[["0","1"]]
query_y = y_query

train_X = pd.read_csv('tsne_X_train.csv')[["0", "1"]]
train_y = y_train

In [33]:
train_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117256 entries, 0 to 117255
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       117256 non-null  float64
 1   1       117256 non-null  float64
dtypes: float64(2)
memory usage: 1.8 MB


In [34]:
"""
    Function that takes in a dataframe of data and outputs 
    a fitted "optimal" model

    Inputs:
    - train: training set
    - query: query set
    - models: dictionary of (model_name : model function) to train and predict on, with optimized 
    parameters already.

    Outputs:
    - best_clf: The optimum classifier function fitted over training data

    - accuracy_score: list of accuracies based on order of models
    passed.
    """

models = {"SVC": post_valid_models[0]}
accuracy_score, best_clf = train_n_predict(train_X, train_y, query_X, query_y, models)

AttributeError: predict_proba is not available when probability=False

In [54]:
leaderboard_path = "../Data/leaderboard_data.csv"
pd.read_csv(leaderboard_path).columns


Index(['ID', 'axonal_coor_x', 'axonal_coor_y', 'axonal_coor_z',
       'dendritic_coor_x', 'dendritic_coor_y', 'dendritic_coor_z', 'adp_dist',
       'post_skeletal_distance_to_soma', 'pre_skeletal_distance_to_soma',
       'pre_oracle', 'pre_test_score', 'pre_rf_x', 'pre_rf_y', 'post_oracle',
       'post_test_score', 'post_rf_x', 'post_rf_y', 'compartment',
       'pre_brain_area', 'post_brain_area', 'pre_nucleus_x', 'pre_nucleus_y',
       'pre_nucleus_z', 'post_nucleus_x', 'post_nucleus_y', 'post_nucleus_z',
       'pre_nucleus_id', 'post_nucleus_id'],
      dtype='object')

In [55]:
sub_data = cleaner(leaderboard_path, feature_path, morph_path, submission = True)
sub_data2 = model.fit_transform(sub_data.select_dtypes(include='number').drop("ID", axis = 1))
ids = pd.read_csv(leaderboard_path)["ID"]
ids

0            0
1            1
2            2
3            3
4            4
         ...  
42588    42588
42589    42589
42590    42590
42591    42591
42592    42592
Name: ID, Length: 42593, dtype: int64

In [56]:
sub_data2=pd.DataFrame(sub_data2)

In [57]:
lb_data = sub_data2.select_dtypes(include='number')

In [58]:
lb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42593 entries, 0 to 42592
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       42593 non-null  float32
 1   1       42593 non-null  float32
dtypes: float32(2)
memory usage: 332.9 KB


In [18]:
# connect_info = data[["ID","ADP_total"]]
# lb_data = lb_data.merge(connect_info, how = 'left', left_on="ID", right_on="ID")
# lb_data.info()

MergeError: Passing 'suffixes' which cause duplicate columns {'connect_rate_x', 'connect_total_x', 'ADP_total_x'} is not allowed.

In [59]:
#create a boolean prediction solution
lb_data["connected"] = post_valid_models[0].predict(lb_data)
lb_data["ID"]=ids

In [60]:
submission_data = lb_data.filter(['ID','connected'])

In [61]:
submission_data.to_csv('submission_data.csv',index=False)